In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

## Reflect Tables into SQLALchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
inspector = inspect(engine)
# reflect the tables
inspector.get_table_names()

In [ ]:
# View all of the classes that automap found
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

In [ ]:
# Create our session (link) from Python to the DB

measurement = Base.classes.measurement
station = Base.classes.station

In [ ]:
session = Session(engine)

In [ ]:
# Save references to each table
measurement_stmt = session.query(measurement).\
    order_by(measurement.id).statement
measurement_df = pd.read_sql_query(measurement_stmt, session.bind)
measurement_df

In [ ]:
station_stmt = session.query(station).\
    order_by(station.id).statement
station_df = pd.read_sql_query(station_stmt, session.bind)
station_df

## Bonus Challenge Assignment: Temperature Analysis II

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, maximum, and average temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(measurement.tobs), func.avg(measurement.tobs), func.max(measurement.tobs)).\
        filter(measurement.date >= start_date).filter(measurement.date <= end_date).all()

# For example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use the function `calc_temps` to calculate the tmin, tavg, and tmax 
# for a year in the data set
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(measurement.tobs), func.avg(measurement.tobs), func.max(measurement.tobs)).\
        filter(measurement.date >= start_date).filter(measurement.date <= end_date).all()

# For example
print(calc_temps('2017-08-01', '2017-08-07'))

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for bar height (y value)
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)
bar_plot = session.query(func.min(measurement.tobs), func.avg(measurement.tobs), func.max(measurement.tobs)).\
        filter(measurement.date >= '2017-08-01').filter(measurement.date <= '2017-08-07').all()

aug_avg = pd.DataFrame(bar_plot, columns=['TMIN', 'TAVE', 'TMAX'])
x= aug_avg['TMAX']
y= aug_avg['TMIN']
z=aug_avg['TAVE']

plot_aug = z.plot.bar(yerr=(x-y), width=0.2, figsize=(3,7), alpha=.6)
plt.ylabel('Temp (F)')
plt.title('Trip Avg Temp')
plt.show()

### Daily Rainfall Average

In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's 
# matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation
total_rain = session.query(measurement.station, station.name, station.latitude, station.longitude, 
                           station.elevation, func.sum(measurement.prcp)).\
        filter(measurement.station == station.station).\
        filter(measurement.date >= '2017-08-01').filter(measurement.date <= '2017-08-07').\
        group_by(measurement.station).\
        order_by(func.sum(measurement.prcp).desc()).all()
total_rain

In [ ]:
# Use this function to calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(measurement.tobs), func.avg(measurement.tobs), func.max(measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", measurement.date) == date).all()

# For example
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`
# Set the start and end date of the trip
start_date = '2017-08-01'
end_date = '2017-08-07'

sel = [func.min(measurement.tobs), func.avg(measurement.tobs), func.max(measurement.tobs),
      func.strftime("%m-%d", measurement.date)]

# Use the start and end date to create a range of dates
start_Date = '08-01'
end_Date = '08-07'

# Strip off the year and save a list of strings in the format %m-%d
# Use the `daily_normals` function to calculate the normals for each date string 
# and append the results to a list called `normals`.

normals_list = session.query(func.min(measurement.tobs), func.avg(measurement.tobs), func.max(measurement.tobs),
      func.strftime("%m-%d", measurement.date)).\
        filter(func.strftime("%m-%d", measurement.date) >= start_Date).\
        filter(func.strftime("%m-%d", measurement.date) <= end_Date).\
        group_by(func.strftime("%m-%d", measurement.date)).statement
normals_list

In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index
normals_df = pd.read_sql_query(normals_list, session.bind)
rename_norm = normals_df.rename(columns={'min_1':'tmin',
                          'avg_1':'tavg',
                          'max_1':'tmax',
                          'strftime_1':'date'})

In [ ]:
clean_norm = rename_norm.set_index('date')
clean_norm

In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
area_plot = clean_norm.plot.area(stacked=False, alpha=.2)
plt.xticks(rotation=45)
plt.grid(True)
plt.title('Daily Temperatures')
plt.xlabel('Date')
plt.ylabel('Tmeperature')
plt.show()

## Close Session

In [ ]:
# Close Session
session.close()